<a href="https://colab.research.google.com/github/pitfisher/miem_face_recognition/blob/main/FR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Эксперимент 2***

*Сравнение лица с его версиями меньшего разрешения Для каждого из изображений лиц создаются его версии в 10 меньших разрешениях: [1024, 768, 512, 256, 224, 128, 112, 96, 64, 32] Далее исходное изображение сравнивается с полученными версиями. Цель: оценить качество работы модели при получении на вход изображения меньшего разрешения чем то, на котором она предобучена. Результаты:*

1. Таблица с дистанциями между исходным и уменьшенным изображениями для каждой персоны в наборе данных
2. Графики с ломанными линиями, показывающие зависимость метрики сходства от разрешения входного изображения
3. Графики рассеяния значений метрик сходства между исходных изображением и его уменьшенными версиями

In [ ]:
!pip install deepface
!pip install matplotlib

In [1]:
import cv2
import time
import os
import glob
from PIL import Image, ImageOps, ImageDraw
from deepface import DeepFace
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

2024-04-24 14:43:22.647210: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 14:43:22.647261: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 14:43:22.647276: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-24 14:43:22.651383: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
input_folder = '/tf/data/dataset'

resolutions = [1024, 768, 512, 256, 224, 128, 112, 96, 64, 32]

In [10]:
model_names = [
    "VGG-Face",
    "Facenet",
    "Facenet512",
    "OpenFace",
    "DeepFace",
    "DeepID",
    "ArcFace",
    "SFace"
]

In [11]:
thresholds = {
     "VGG-Face": 1.17,
    "Facenet": 0.8,
    "Facenet512": 1.04,
    "OpenFace": 0.55,
    "DeepFace": 0.64,
    "DeepID": 0.17,
    "ArcFace": 1.13,
    "SFace": 1.055,
}

In [12]:
def crop_to_square_pil(image_array, face_info, border_pixels=0):
    
    image = Image.fromarray(image_array)
    x, y, w, h = face_info['facial_area']['x'], face_info['facial_area']['y'], face_info['facial_area']['w'], face_info['facial_area']['h']
    square_size = max(w, h) + 2 * border_pixels
    new_x = max(0, x)
    new_y = max(0, y)
    cropped_image = image.crop((new_x - (h-w)//2 - border_pixels, new_y - border_pixels, new_x - (h-w)//2 + square_size, new_y + square_size))

    return cropped_image

In [13]:
def resize_image_pil(image, new_resolution):
    resized_img = image.resize(new_resolution, Image.LANCZOS)
    return resized_img

In [14]:
def save_to_csv(data, columns, filename):
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(filename + '.csv', index=False)

In [ ]:
files = glob.glob(os.path.join(input_folder, "**", "*.JPG"), recursive=True)

data = {}

for model_name in model_names:

    for img_path in tqdm(files, desc=f"Processing images with {model_name}", unit="image"):

        path = img_path.split(os.path.sep)
        filename = f'{path[-2]}_{path[-1]}'
        ds_name = f'{model_name}_{path[-4]}_{path[-3]}'

        try:
            face_info = DeepFace.extract_faces(img_path=img_path, detector_backend="retinaface")[0]
        except ValueError as e:
            print(f"Face detection failed {img_path}: {e}")
            continue

        image = Image.open(img_path)
        image = ImageOps.exif_transpose(image)
        image_array = np.array(image)
        
        cropped_img = crop_to_square_pil(image_array, face_info)
        img1 = np.array(cropped_img)

        metrics = [] 
 
        for resolution in resolutions:

            resized_img = resize_image_pil(cropped_img, (resolution, resolution))
            img2 = np.array(resized_img)
            start_time = time.perf_counter()

            try:
                result = DeepFace.verify(img1_path=img1,
                                         img2_path=img2,
                                         enforce_detection=False,
                                         detector_backend="retinaface",
                                         model_name=model_name,
                                         distance_metric="euclidean_l2"
                                         )
                end_time = time.perf_counter()
                fin_time = end_time - start_time

                metrics.append(f'{result["distance"]:.5f}')
                metrics.append(f'{fin_time:.5f}')

            except ValueError as e:
                print(f"Verification failed {img_path}: {e}")
                metrics.extend([None, None])

        if ds_name not in data:
            data[ds_name] = []

        data[ds_name].append([filename] + metrics)

columns = ['name'] + [f"{res}_{metric}" for res in resolutions for metric in ["distance", "time"]]

for ds_name, ds in data.items():
    save_to_csv(ds, columns, ds_name)

Processing images with VGG-Face:   0%|          | 0/633 [00:00<?, ?image/s]